# Imports

# Backgrounds

This tutorial is based on the paper Bayesian Parameter Estimation using Conditional Variational Autoencoders for Gravitational-wave Astronomy (https://arxiv.org/abs/1909.06296). In this demo, we will show how a form of conditional variational autoencoders work and how to reproduce Bayesian posteriors several orders of magnitude faster than exisitng Bayesian techniques.

# Generate Bilby Posteriors

## Define bounds and default fixed values for source parameters and search

## Generate Requested number of testing samples

# Intro to VItamin

# Generate Traning Samples

In [0]:
ls

sample_data/


# Compare Bilby vs. VItamin

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Further Reading